In [ ]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import sys

import sys
sys.path.insert(0,'source_data/arcplot-main/')
from arcplot import ArcDiagram


def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [ ]:
major_brain_regions = ['LSX_HY_MB_HB',
 'FbTrt',
 'CB_1',
 'CB_2',
 'MYdp',
 'Hbl_VS',
 'L1_HPFmo_Mngs',
 'TH',
 'CTX_1',
 'CTX_2',
 'ENTm',
 'DG',
 'HPF_CA',
 'HY',
 'OB_1',
 'STR',
 'OB_2']

color_tissue=pd.read_csv('source_data/color/starmap_sub.csv')
dic_color={}
for key,color in zip(color_tissue['key'],color_tissue['color']):
    dic_color[key]=color
    

In [ ]:

for atlas_source in ['','_Atlas1','_Atlas2','_Atlas3']:
    permutation_path_merfish=f'source_data/outputs_30um{atlas_source}'
    combined_results=pd.read_csv(os.path.join(permutation_path_merfish+'/result/', 'all_close_contacts.csv'))
    save_path=os.path.join(permutation_path_merfish+'/figures_pvalue/')
    os.makedirs(save_path, exist_ok=True)

    for region_i in major_brain_regions:

        combined_results_CB=combined_results.loc[combined_results['region']==region_i]
        if combined_results_CB.shape[0]<2:
            continue
#         print(region_i)
        combined_results_CB['from'] = combined_results_CB['cell_type1']
        combined_results_CB['to'] = combined_results_CB['cell_type2']

        pvalue=combined_results_CB['z_score']
        combined_results_CB['weights'] = pvalue 

        transparency=np.array(-np.log10(combined_results_CB['pval-adjusted']))+10
        transparency[transparency==np.inf]=350
        transparency = NormalizeData(transparency) 
        transparency[transparency<0.7]=0.7
        combined_results_CB['alpha'] = transparency


        combined_results_CB_link =combined_results_CB[['from','to','weights','alpha',]].copy()


        def createArcDiagram(df, node1, node2, weights=None, alpha=None,
                             bg_color='white',
                             cmap='viris', title=f'Diagram of {region_i} {atlas_source}'):

            # get all the nodes
            nodes_old = df[node1].unique().tolist() + df[node2].unique().tolist()
            nodes_old = list(set(nodes_old))

            nodes=[]
            for i in dic_color.keys():
                if i in nodes_old:
                    nodes.append(i)

            custom_colors=[dic_color[i] for i in nodes]
            # create the diagram
            arcdiag = ArcDiagram(nodes, title)
            arcdiag.set_custom_colors(custom_colors)
            arcdiag.set_label_rotation_degree(90)
            if not weights:
                df['weights'] = 0.1

            # connect the nodes
            for connection in df.iterrows():
                arcdiag.connect(
                    connection[1][node1],
                    connection[1][node2],
                    transparency=connection[1][alpha],
                    linewidth=connection[1][weights],
                    )

            # custom colors
            arcdiag.set_background_color(bg_color)

            # plot the diagram
            # arcdiag.show_plot()
            arcdiag.save_plot_as(f'{save_path}/{region_i}.png', resolution=300) # for saving file as an image with an optional resolution setting for higher-quality images.


        createArcDiagram(
            combined_results_CB_link,
            node1='from',
            node2='to',
            weights='weights',
            alpha='alpha',
        )
        # break
    